In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, AveragePooling2D, Flatten, Dense, MaxPool2D, concatenate, MaxPooling2D # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore

In [2]:
data = np.load('oxflower17.npz')

In [3]:
x_train = data['X']
y_train = data['Y']

In [4]:
x_train.shape, y_train.shape

((1360, 224, 224, 3), (1360, 17))

In [5]:
# Inception Block
def inception_block(x, filters):
    tower_1 = Conv2D(filters[0], (1,1), padding = 'same', activation = 'relu')(x)
    tower_1 = Conv2D(filters[1], (3,3), padding = 'same', activation = 'relu')(tower_1)
    
    tower_2 = Conv2D(filters[2], (1,1), padding = 'same', activation = 'relu')(x)
    tower_2 = Conv2D(filters[3], (3,3), padding = 'same', activation = 'relu')(tower_2)
    
    tower_3 = MaxPooling2D((3,3), strides = (1,1), padding = 'same')(x)
    tower_3 = Conv2D(filters[4], (1,1), padding = 'same', activation = 'relu')(tower_3)
    
    output = concatenate([tower_1, tower_2, tower_3], axis = 3)
    return output

# Build the inception model
def inception(input_shape, num_classes):
    inputs = keras.Input(shape = input_shape)
    
    x = Conv2D(64, (3,3), padding = 'same', activation = 'relu')(inputs)
    x = MaxPooling2D((2,2))(x)
    
    x = inception_block(x, filters = [64, 94, 128, 16, 32])
    x = inception_block(x, filters = [128, 128, 192, 32, 96])
    x = MaxPooling2D((2,2))(x)
    
    x = inception_block(x, filters = [192, 96, 208, 16, 48])
    x = inception_block(x, filters = [160, 112, 224, 24, 64])
    x = inception_block(x, filters = [128, 128, 256, 24, 64])
    x = inception_block(x, filters = [112, 144, 288, 32, 64])
    x = MaxPooling2D((2,2))(x)
    
    x = inception_block(x, filters = [256, 160, 320, 32, 128])
    x = inception_block(x, filters = [256, 160, 320, 32, 128])
    x = inception_block(x, filters = [384, 192, 384, 48, 128])
    
    x = AveragePooling2D((4,4))(x)
    x = Flatten()(x)
    outputs = Dense(num_classes, activation = 'softmax')(x)
    
    model = Model(inputs = inputs, outputs = outputs)
    return model
    

In [6]:
model = inception(input_shape = (224,224,3), num_classes = 17)

model.compile(optimizer = Adam(learning_rate=0.001), loss = "categorical_crossentropy", metrics = ['accuracy'])

print(model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 224, 224,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 112, 112,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 112, 112,  │      4,160 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 112, 112,  │      8,320 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 112, 112,  │          0 │ max_pooling2d[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 112, 112,  │     54,238 │ conv2d_1[0][0]    │
│                     │ 94)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 112, 112,  │     18,448 │ conv2d_3[0][0]    │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 112, 112,  │      2,080 │ max_pooling2d_1[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 112, 112,  │          0 │ conv2d_2[0][0],   │
│ (Concatenate)       │ 142)              │            │ conv2d_4[0][0],   │
│                     │                   │            │ conv2d_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 112, 112,  │     18,304 │ concatenate[0][0] │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 112, 112,  │     27,456 │ concatenate[0][0] │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 112, 112,  │          0 │ concatenate[0][0] │
│ (MaxPooling2D)      │ 142)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 112, 112,  │    147,584 │ conv2d_6[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 112, 112,  │     55,328 │ conv2d_8[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 112, 112,  │     13,728 │ max_pooling2d_2[… │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 4,307,855 (16.43 MB)

 Trainable params: 4,307,855 (16.43 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
with tf.device('/GPU:0'):
    model.fit(x_train, y_train, batch_size=64, epochs=5, verbose=1,validation_split=0.2, shuffle=True)

Epoch 1/50
